297. Serialize and Deserialize Binary Tree

Hard

Companies
Serialization is the process of converting a data structure or object into a sequence of bits so that it can be stored in a file or memory buffer, or transmitted across a network connection link to be reconstructed later in the same or another computer environment.

Design an algorithm to serialize and deserialize a binary tree. There is no restriction on how your serialization/deserialization algorithm should work. You just need to ensure that a binary tree can be serialized to a string and this string can be deserialized to the original tree structure.

---

Observations:
- not a BST (not that this really matters)
- must be able to determine separate node.vals of ranging lengths (i.e. 1 vs 10 vs 100 vs 1000). This matters becuase 1234 could be 1,2,3,4 or 12,34 or 123,4 etc. Therefore, must use comma delimination to identify values in the string. 

First Solution:
Create two separate strings that represent the inorder and preorder traversals of the tree. Then combine those traversals into a string that can be decoded using a deliminator like "#". 

In [ ]:
class Codec:
    def preorder(self, root):
        # creates an array of the preorder traversal of a given tree
        preorder = []
        def dfs(root):
            if not root:
                preorder.append("N")
                return
            else:
                val = str(root.val)
            preorder.append(val)
            dfs(root.left)
            dfs(root.right)
        dfs(root)
        return preorder

    def inorder(self,root):
        res,stack = [], []
        cur = root
        while cur or stack:
            # goes as left as possible
            while cur:
                stack.append(cur)
                cur = cur.left
            cur = stack.pop()
            res.append(str(cur.val))
            cur = cur.right
        return res

    def constructTree(self, preorder, inorder):
        if len(inorder) == 0:
            return None
        # else we have a valid
        val = preorder.pop(0)
        index = inorder.index(val) 
        root = TreeNode(int(val))
        root.left = self.constructTree(preorder,inorder[0:index])
        root.right = self.constructTree(preorder,inorder[index + 1:])
        return root
    
    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        preorder = self.preorder(root)
        inorder = self.inorder(root)
        strPre = ",".join(preorder)
        strIn = ",".join(inorder)
        encoded = strPre + "#" + strIn
        return encoded
        

    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        strPre, strIn = data.split('#')
        listPre = strPre.split(",")
        listIn = strIn.split(",")
        if len(listPre) == 0:
            return None
        tree = self.constructTree(listPre,listIn)
        return tree

This approach does not work when there are multiple instances of the same value in the tree. Otherwise it does work. Therefore, I need to make a huge change to the code.

I am still using preorder traversal. However, we ARE recording where the null nodes exist. This will allow us the ability to deconstruct the encoded string into something we can work with. My original solution was a good attempt, but as always with a leetcode problem, far too many lines of code, and a far simplier answer exits. I am still proud of myself for getting something that mostly work together. 

The challange now that I have the encoded string is the decoding process.

In [ ]:
class Codec:
    def constructTree(self, preorder, inorder):
        if len(inorder) == 0:
            return None
        # else we have a valid
        print("construct",preorder,inorder)
        val = preorder.pop(0)
        index = inorder.index(val) 
        root = TreeNode(int(val))
        root.left = self.constructTree(preorder,inorder[0:index])
        root.right = self.constructTree(preorder,inorder[index + 1:])
        return root
    
    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        preorder = []
        def dfs(root):
            if not root:
                preorder.append("N")
                return
            else:
                val = str(root.val)
            preorder.append(val)
            dfs(root.left)
            dfs(root.right)
        dfs(root)
        strPre = ",".join(preorder)
        return strPre
        

    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        listPre = data.split(",")
        if listPre[0] == '':
            return None
        def dfs(preorder):
            val = preorder.pop(0)
            if val == "N":
                return None
            root = TreeNode(int(val))
            root.left = dfs(preorder)
            root.right = dfs(preorder)
            return root
        tree = dfs(listPre)
        return tree